In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
import re
import functools
import string
import pandas as pd
import numpy as np
import seaborn as sns
import sqlite3
import time
pd.set_option('display.max_rows', 300)

In [ ]:
dir_path = '/content/drive/MyDrive/CSS_project/data/'
con = sqlite3.connect(dir_path+"queen.db", timeout=10)

In [ ]:
con.execute("""SELECT name FROM sqlite_master WHERE type='table';""").fetchall()

[('Sentiment',),
 ('Tweets',),
 ('Retweets',),
 ('Authors',),
 ('Gender',),
 ('Genders',),
 ('Authors_country',)]

In [ ]:
# con.execute("PRAGMA table_info('Authors')").fetchall()

In [ ]:
authors_with_loc = con.execute('select author_id, location from authors').fetchall()

In [ ]:
authors_with_loc = pd.DataFrame(authors_with_loc).rename(columns={0:'author_id',1:'location'})

In [ ]:
authors_with_loc.head()

,author_id,location
0,1485659018626535433,she/her
1,4272845427,Islamic Republic of Pakistan.
2,1176370519437795329,pan🏳️‍🌈
3,2426367270,
4,1486740983454871562,Tulancingo Hidalgo México


In [ ]:
df_loc = pd.read_csv(dir_path+'from_author_location_to_country.csv', index_col=0)

In [ ]:
df_loc.head(3)

,location,count,country
0,"London, England",20712,GB
1,United States,13587,US
2,United Kingdom,12982,GB


In [ ]:
authors_country = authors_with_loc.join(df_loc.set_index('location').drop(columns='count'), on='location')

In [ ]:
authors_country.head(3)

,author_id,location,country
0,1485659018626535433,she/her,NaN
1,4272845427,Islamic Republic of Pakistan.,NaN
2,1176370519437795329,pan🏳️‍🌈,NaN


In [ ]:
authors_country_not_null = authors_country[~authors_country.country.isna()].drop(columns='location')

In [ ]:
authors_country_not_null.head(3)

,author_id,country
7,1469681896854597645,BR
9,1368266098508959744,US
12,1128384987110281222,CA


# create a table with author_id and country

In [ ]:
# con.execute("""DROP TABLE IF EXISTS Authors_country""")
# con.execute("""CREATE TABLE IF NOT EXISTS Authors_country (
#             author_id TEXT NOT NULL PRIMARY KEY UNIQUE,
#             country TEXT NOT NULL);
# """)

In [ ]:
# con = sqlite3.connect(dir_path+"queen.db", timeout=10)
# for i, (a_id, c_code) in enumerate(authors_country_not_null.values):
#   print(f'Inserting entry #{i}')
#   if not i%500: # commit every 200 entries
#     con.commit()
#     con = sqlite3.connect(dir_path+"queen.db", timeout=10)
#   con.execute('''INSERT INTO Authors_country VALUES (?,?)''', (a_id, c_code))
# con.commit()

In [ ]:
con.execute('select count(*) from authors_country').fetchall()

[(885057,)]

# load country into the authors table

In [ ]:
# con.execute('ALTER TABLE authors ADD country TEXT')

In [ ]:
# con.execute('''UPDATE Authors SET country = (SELECT Authors_country.country 
#                          FROM Authors_country 
#                          WHERE Authors_country.author_id = Authors.author_id)''')

# Get all tweet for which country is available, along with text and country

In [ ]:
con.execute("PRAGMA table_info('Tweets')").fetchall()

[(0, 'tweet_id', 'TEXT', 1, None, 1),
 (1, 'author_id', 'TEXT', 1, None, 0),
 (2, 'tweet_text', 'TEXT', 1, None, 0),
 (3, 'country', 'TEXT', 0, None, 0),
 (4, 'retweet_count', 'INTEGER', 0, None, 0),
 (5, 'reply_count', 'INTEGER', 0, None, 0),
 (6, 'like_count', 'INTEGER', 0, None, 0),
 (7, 'quote_count', 'INTEGER', 0, None, 0)]

In [ ]:
con.execute("PRAGMA table_info('Retweets')").fetchall()

[(0, 'tweet_id', 'TEXT', 1, None, 1),
 (1, 'original_tweet_id', 'TEXT', 1, None, 0),
 (2, 'author_id', 'TEXT', 1, None, 0),
 (3, 'original_author_id', 'TEXT', 1, None, 0),
 (4, 'country', 'TEXT', 0, None, 0),
 (5, 'retweet_count', 'INTEGER', 0, None, 0),
 (6, 'reply_count', 'INTEGER', 0, None, 0),
 (7, 'like_count', 'INTEGER', 0, None, 0),
 (8, 'quote_count', 'INTEGER', 0, None, 0)]

In [ ]:
# con.execute("PRAGMA table_info('Authors')").fetchall()
con.execute("PRAGMA table_info('Authors_country')").fetchall()

[(0, 'author_id', 'TEXT', 1, None, 1), (1, 'country', 'TEXT', 1, None, 0)]

In [ ]:
tweets_retrieved = []

In [ ]:
# get tweet_id for which country is available in the tweet object
results = con.execute('''select tweet_id, tweet_text, country from tweets where country!='' ''').fetchall()
results = list(results)
tweet_ids_from_tweets = {r[0] for r in results}
country_names = {r[2] for r in results}
len(tweet_ids_from_tweets)

15583

convert country name to alpha2 code

In [ ]:
!pip3 install pyco?untry
import pycountry
countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_2

ERROR: Invalid requirement: 'pyco?untry'


In [ ]:
codes = {c:countries.get(c, 'Unknown') for c in country_names}

In [ ]:
codes['Republic of the Philippines'] = 'PH'
codes['Republic of Korea'] = 'KR'
codes['Bolivia'] = 'BO'
codes['Republic of Mauritius'] = 'MU'
codes['Vietnam'] = 'VN'
codes['Democratic Republic of Congo'] = 'CD'
codes['Islamic Republic of Iran'] = 'IR'
codes['Former Yugoslav Republic of Macedonia'] = 'MK'
codes['Pitcairn Islands'] = 'PN'
codes['Republic of Slovenia'] = 'SI'
codes['Venezuela'] = 'VE'
codes['The Netherlands'] = 'NL'
codes['Hashemite Kingdom of Jordan'] = 'JO'
codes["People's Republic of China"] = 'CN'
codes['Saint Helena, Ascension and Tristan Da Cunha'] = 'SH'
codes['Macau'] = 'MO'
codes['Republic of Mozambique'] = 'MZ'
codes['Republic of Serbia'] = 'RS'
codes['Kosovo'] = 'XK'
codes['Norfolk Islands'] = 'NF'
codes['Kingdom of Saudi Arabia'] = 'SA'
codes['Taiwan'] = 'TW'
codes['Brunei'] = 'BN'
codes['Republic of Croatia'] = 'HR'
codes['Tanzania'] = 'TZ'
codes['Czech Republic'] = 'CZ'

In [ ]:
temp = [(r[0], r[1], codes[r[2]]) for r in results]
tweets_retrieved += temp

In [ ]:
# get all authors with country
results = con.execute('''select author_id, country from authors_country''').fetchall()
authors_with_country = list(results)
authors_with_country = {k:v for k,v in authors_with_country}
a_id = {r[0] for r in results}

In [ ]:
# get tweet_id from tweets for which country is identified via author location
results_all = con.execute('''select tweet_id, tweet_text, author_id from tweets''').fetchall()
results_all = list(results_all)
id_from_author_location = {r[0] for r in results_all if r[2] in a_id}.difference(tweet_ids_from_tweets)

In [ ]:
temp = [t for t in results_all if t[0] in id_from_author_location]
temp = [(t[0], t[1], authors_with_country[t[2]]) for t in temp]
len(temp)

295444

In [ ]:
tweets_retrieved += temp
len(tweets_retrieved)

311027

In [ ]:
all_tweets_df = pd.DataFrame(tweets_retrieved).rename(columns={0:'tweet_id',1:'tweet_text',2:'country'})

In [ ]:
all_tweets_df

,tweet_id,tweet_text,country
0,1572368735440183297,Its nice that India waited until after the que...,IN
1,1572368364311564288,@CroweDM Qs1. Are you sad the queen died\nQs...,AU
2,1572367957367349248,Is only Ghana interview that they will asked y...,GH
3,1572367617146630145,@SarahLGates1 @amz360 The queen is dead and so...,AU
4,1572361653458505728,"Nutball did not believe the Queen was dead, w...",GB
...,...,...,...
311022,1566949797608710145,@FakeDawnSummers then shampoo commercial versi...,US
311023,1566947257856966656,Queen of the Death Match.,US
311024,1566943573316075520,rare photo of Queen Elizabeth bashing a coati ...,US
311025,1566940632698044422,All of that just for him to scream at a dead d...,GB


In [ ]:
# all_tweets_df.to_csv(dir_path+'all_tweets_with_country.csv', index=False)

In [ ]:
# all_tweets_df = pd.read_csv(dir_path+'all_tweets_with_country.csv')

Text of retweets are all in tweets table, so we do not need to make separate inferences for them.

# EmoRoBERTa for emotional analysis
Trained on reddit, no proper performance report

In [ ]:
# !pip3 install transformers

In [ ]:
# from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline
# tokenizer = RobertaTokenizerFast.from_pretrained("arpanghoshal/EmoRoBERTa")
# model = TFRobertaForSequenceClassification.from_pretrained("arpanghoshal/EmoRoBERTa")
# emotion = pipeline('sentiment-analysis', model='arpanghoshal/EmoRoBERTa', return_all_scores=True)
# emotion('Hello')

In [ ]:
samples = con.execute('select tweet_text from tweets limit 1000').fetchall()
samples = [s[0] for s in samples]

In [ ]:
# test = [emotion(s) for s in samples]
# test = [{x['label']:x['score'] for x in t[0]} for t in test]

# Another BERT trained on twitter sentiment data 
Trained on twitter sentiment data

https://huggingface.co/bhadresh-savani/bert-base-uncased-emotion?text=I+like+you.+I+love+you&fbclid=IwAR2LwVPAZA6eSuumf8EqJpTDitfVTIzX5ks009C-EMa4YP88bCezbKmecP8

In [ ]:
import json

In [ ]:
import pandas as pd
dir_path = '/content/drive/MyDrive/CSS_project/data/'

In [ ]:
# # !pip3 install transformers
# from transformers import pipeline
# classifier = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True) # the smaller model

In [ ]:
df = pd.read_csv(dir_path+'all_tweets_with_country.csv').dropna().reset_index(drop=True)
# samples = df[:100]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
# prediction = classifier(list(samples['tweet_text']))
# prediction = classifier(list(df['tweet_text']))

In [ ]:
# cols = ['sadness','joy','love','anger','fear','surprise']
# sent_scores = [[sent['score'] for sent in p] for p in prediction]

In [ ]:
# df_sent = pd.DataFrame(sent_scores).rename(columns={i:c for i,c in enumerate(cols)})

In [ ]:
# tweets_with_sent = df.join(df_sent)

In [ ]:
# tweets_with_sent.to_csv('BACKUP_SENTIMENT.csv', index=False)

In [ ]:
df_minority = df[(df.country!='US')&(df.country!='GB')&(df.country!='CA')&(df.country!='AU')]

In [ ]:
df_minority.shape

(81273, 3)

In [ ]:
# with open(dir_path+'NON_US_UK_CA_AU_SENTIMENT.json','w') as f:
#   j=0
#   for i, t, c in df_minority.values[:20]:
#     if not j%100:
#       print(j)
#     pred = classifier(t)
#     sent_scores = {i:{'tweet_text': t, 'country':c, 'sentiments':{s['label']:s['score'] for s in pred[0]}}}
#     sent_scores_string = json.dumps(sent_scores)
#     f.write(sent_scores_string+'\n')
#     j+=1

0


In [ ]:
len(df)

311012

# This has tweets, country and sentiment scores

In [ ]:
df_all = pd.read_csv(dir_path + 'all_tweets_with_country_and_sentiments.csv', index_col=0)

In [ ]:
df_all.head()

,tweet_id,tweet_text,country,sentiments,sadness,joy,love,anger,fear,surprise,emotion
0,1572368735440183297,Its nice that India waited until after the que...,IN,"{'sadness': 0.7237589359283447, 'joy': 0.11996...",0.723759,0.119966,0.008417,0.141892,0.004593,0.001373,sadness
1,1572368364311564288,@CroweDM Qs1. Are you sad the queen died\nQs...,AU,"{'sadness': 0.07540038973093033, 'joy': 0.9146...",0.075400,0.914668,0.003434,0.004386,0.000891,0.001220,joy
2,1572367957367349248,Is only Ghana interview that they will asked y...,GH,"{'sadness': 0.0072736977599561214, 'joy': 0.07...",0.007274,0.071170,0.002253,0.579968,0.336136,0.003200,anger
3,1572367617146630145,@SarahLGates1 @amz360 The queen is dead and so...,AU,"{'sadness': 0.7509899735450745, 'joy': 0.01143...",0.750990,0.011435,0.001622,0.062640,0.170549,0.002764,sadness
4,1572361653458505728,"Nutball did not believe the Queen was dead, w...",GB,"{'sadness': 0.0754714235663414, 'joy': 0.00735...",0.075471,0.007355,0.000935,0.115649,0.798253,0.002336,fear


# Need author ids to join

In [ ]:
con = sqlite3.connect(dir_path+"queen.db", timeout=10)
results = con.execute('''select tweet_id, author_id from tweets''').fetchall()
results = list(results)
tid_to_aid = {tid:aid for tid,aid in results}

In [ ]:
df_all['author_id'] = df_all.tweet_id.apply(str).apply(lambda x: tid_to_aid[x]) # get author id

In [ ]:
df_all.head(2)

,tweet_id,tweet_text,country,sentiments,sadness,joy,love,anger,fear,surprise,emotion,author_id
0,1572368735440183297,Its nice that India waited until after the que...,IN,"{'sadness': 0.7237589359283447, 'joy': 0.11996...",0.723759,0.119966,0.008417,0.141892,0.004593,0.001373,sadness,1566773297798434818
1,1572368364311564288,@CroweDM Qs1. Are you sad the queen died\nQs...,AU,"{'sadness': 0.07540038973093033, 'joy': 0.9146...",0.075400,0.914668,0.003434,0.004386,0.000891,0.001220,joy,1473605424813203458


# Add ethnicity and gender

# Add gender

In [ ]:
all_gender = pd.read_csv(dir_path+'gender.csv')
all_gender['gender'] = all_gender.gender_score.apply(lambda x: 'F' if x>0.5 else 'M')
all_gender['author_id'] = all_gender.author_id.apply(str)

In [ ]:
df_all = df_all.join(all_gender[['author_id','gender']].set_index('author_id'), on='author_id')

In [ ]:
# df_all.to_csv(dir_path+'all_tweets_with_country_sentiments_gender.csv')

# Add ethnicity
The order of the authors are the same as that in the db

In [ ]:
df_all = pd.read_csv(dir_path+'all_tweets_with_country_sentiments_gender.csv', index_col=0)

In [ ]:
with open(dir_path+'ethnicity.json', 'r') as f:
  all_ethnicity = f.readlines()
all_ethnicity = all_ethnicity[0]
all_ethnicity = json.loads(all_ethnicity)

In [ ]:
relevant_author_ids = set(df_all.author_id)
len(relevant_author_ids)

198103

In [ ]:
con = sqlite3.connect(dir_path+"queen.db", timeout=10)
r = list(con.execute('''select author_id from authors''').fetchall())
r = [x[0] for x in r]

In [ ]:
temp = {a_id: all_ethnicity[str(i)] for i, a_id in enumerate(r)}

In [ ]:
eth_df = pd.DataFrame.from_dict(temp, orient='index').reset_index().rename(columns={'index':'author_id'})

In [ ]:
eth_df = eth_df[eth_df.author_id.isin(relevant_author_ids)] # filter out authors not present in the big table

In [ ]:
eth_df = eth_df.replace({'(S)':0}).fillna(-1)

In [ ]:
eth_list = ['pctwhite','pctblack','pctapi','pctaian','pct2prace','pcthispanic']
for e in eth_list:
  eth_df[e] = eth_df[e].apply(float)

In [ ]:
eth_df['ethnicity'] = np.nan

In [ ]:
eth_df.loc[eth_df[eth_df['pctwhite']==-1].index, 'ethnicity']='NOT_FOUND'

In [ ]:
temp = eth_df[eth_df['pctwhite']!=-1]
eth_df.loc[temp.index, 'ethnicity']= temp[eth_list].idxmax(axis=1)

In [ ]:
eth_df.ethnicity.value_counts(normalize=True)

NOT_FOUND      0.576069
pctwhite       0.313499
pctapi         0.051726
pcthispanic    0.035830
pctblack       0.019424
pctaian        0.003332
pct2prace      0.000121
Name: ethnicity, dtype: float64

author_id
1000004655433289728     pctwhite
1000006565909409798     pctwhite
100001002              NOT_FOUND
1000020055973945344     pctwhite
1000023473035628545     pctwhite
                         ...    
999946096972615681      pctwhite
999947450378014721     NOT_FOUND
999958937368780801      pctwhite
999969554276212739     NOT_FOUND
999989341               pctwhite
Name: ethnicity, Length: 198103, dtype: object

In [ ]:
df_all_with_eth = df_all.join(eth_df.set_index('author_id')['ethnicity'], on='author_id')

In [ ]:
df_all_with_eth.ethnicity.value_counts(normalize=True)

NOT_FOUND      0.572332
pctwhite       0.316557
pctapi         0.050757
pcthispanic    0.033655
pctblack       0.023758
pctaian        0.002820
pct2prace      0.000122
Name: ethnicity, dtype: float64